In [ ]:
%matplotlib inline

import os

import mne
import numpy as np

## Epoching timeseries data in MNE - the `Epochs` class

For many analyses, it is useful to divide timeseries data into discrete chunks of time, called [epochs](https://mne.tools/stable/documentation/glossary.html#term-epochs).

Epochs can take the form of individuals trials (e.g. isolating data around a given stimulus or behaviour), or divide continuous resting-state data into discrete chunks.

Epochs are stored in MNE as [`mne.Epochs`](https://mne.tools/stable/generated/mne.Epochs.html) objects.

### Part 1 - Epoching data from events

To explore how we can create epochs around events (e.g. stimulus presentation, behaviour), we will reload the example dataset and isolate the EEG data and stimulus channels.

In [ ]:
# Load sample data from disk
raw = mne.io.read_raw_fif(
    os.path.join(mne.datasets.sample.data_path(), "MEG", "sample", "sample_audvis_raw.fif")
)

# Select only EEG and stimulus channels
raw.pick(["eeg", "stim"])
raw.info

Stimulus channels contain information about e.g. when stimuli were presented to subjects, when subjects performed an action, etc...

We can use the `plot()` method to visualise how stimulus data is stored in the `Raw` object.

In [ ]:
# Plot data of stimulus channels
raw.copy().pick("stim").plot();

The [`mne.find_events()`](https://mne.tools/stable/generated/mne.find_events.html) function can be used to convert this information into discrete timepoints based on changes in the signal.

[Events](https://mne.tools/stable/documentation/glossary.html#term-events) are stored as an array of shape `(events, 3)`, where:
- the first column is the timepoint of the event (in samples)
- the second column is the previous type of the event 
- the third column is the new type of the event

An event ID of `0` corresponds to the absence of an event, and event IDs > `0` are stimuli/responses.

In [ ]:
# Find the events from a given stimulus channel
events = mne.find_events(raw, stim_channel="STI 014")

# Print a subset of events
events[:5]

Using these events, we can now create an `Epochs` object.

If we already have a `Raw` object, this is simply a case of passing the `Raw` object and the events array to the `Epochs` class.

Here, we create epochs for all events with an ID > `0`, taking the data from 1 second before to 1 second after each event using the `tmin` and `tmax` parameters (times are relative to the timings of events).

In [ ]:
# Epoch timeseries data from event markers
epochs = mne.Epochs(raw=raw, events=events, tmin=-1, tmax=1)
epochs

As you can see:
-   this data has 320 events across all event types.
-   we have selected data in the [-1, +1] second window around each event.
-   each epoch was baseline-corrected using the time from the start of each epoch to the event itself (0 seconds).

Similarly to `Raw` objects, we can visualise the data stored in `Epochs` objects using the [`plot()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.plot) method.

In [ ]:
# Plot first 3 epochs in the data
epochs.plot(scalings="auto", n_epochs=3);

As for `Raw` objects, the data itself can be accessed using the [`get_data()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.get_data) method, which returns an array of shape `(epochs, channels, times)`.

Having the epochs as the first dimension is convenient for iterating over the data of each epoch, e.g.
```python
    data = epochs.get_data()
    for epoch_data in data:
        ### Do something with the data of a single epoch...
```

In [ ]:
# Get data as an array
data = epochs.get_data(copy=False)
print(f"Data has shape: {data.shape} (epochs, channels, times)")

**Exercises - Creating epochs around events**

**Exercise:** Create epochs around all events in the window [-2, +2] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around all events in the window [-1, +3] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around all events in the window [-0.5, +0.5] seconds.

In [ ]:
## CODE GOES HERE

Baseline-correction of epochs involves taking the mean of a given data period and subtracting this value from each data point of the whole epoch.

The `baseline` parameter of `Epochs` is used to control which period is used for baseline correction.

The default when creating an `Epochs` object in MNE is to take the period from the start of the epoch to the event itself as the baseline period, specified as `baseline=(None, 0)`.

Like for `tmin` and `tmax`, the times in `baseline` are relative to the events.	

**Exercise:** Create epochs around all events in the window [-1, +1], but only use the window [-0.5, 0] seconds as a baseline.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around all events in the window [-1, +2], but only use the window [0, +2] seconds as a baseline.

There are two ways you can specify this, so try to include them both.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around all events in the window [-1, +2], and use this whole period as a baseline.

There are again two ways you can specify this, so try to include them both.

In [ ]:
## CODE GOES HERE

Above, we have been creating epochs around all events, however we may wish to only create epochs around a single type of event.

Epochs for only particular event types can be specified using the `event_id` parameter of `Epochs`.

**Exercise:** Create epochs around only the events with an ID of `1`.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around only the events with an ID of `2`.

In [ ]:
## CODE GOES HERE

**Exercise:** Create epochs around only the events with IDs of `1`, `2`, and `3`.

In [ ]:
## CODE GOES HERE

As you can see, the `Epochs` object of MNE is a very convenient way to create epochs of a given duration, with a given baseline, around specific stimuli/behaviours.

However, we can also create continuous epochs unrelated to any events, such as you would do for an analysis of resting-state data (i.e. no stimuli, no behaviour).

### Part 2 - Creating continuous epochs of data

Continuous epochs can be created easily from `Raw` object using the [`mne.make_fixed_length_epochs()`](https://mne.tools/stable/generated/mne.make_fixed_length_epochs.html) function.

This creates an `Epochs` object with epochs of a specified duration directly from a `Raw` object.

Here we create an `Epochs` object with 1-second-long epochs.

In [ ]:
# Create continuous epochs
epochs = mne.make_fixed_length_epochs(raw=raw, duration=1)
epochs.plot(scalings="auto", n_epochs=3)
epochs

**Exercises - Creating continuous epochs (specifying the duration)**

**Exercise:** Create an `Epochs` object with 2-second-long epochs and verify the length of epochs.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `Epochs` object with 4-second-long epochs and verify the length of epochs.

In [ ]:
## CODE GOES HERE

Continuous epochs do not need to contain data for unique windows of data.

We can artificially increase the amount of data available by having overlapping epochs. By default, there is no overlap between epochs.

Here, we create 1-second-long epochs that have an overlap of 0.5 seconds (50% overlap).

How does the number of epochs compare to that when there was no overlap?

In [ ]:
# Create continuous epochs with overlap
epochs = mne.make_fixed_length_epochs(raw=raw, duration=1, overlap=0.5)
epochs.plot(scalings="auto", n_epochs=3)
epochs

**Exercises - Creating continuous epochs (specifying the overlap)**

**Exercise:** Create 2-second-long epochs with 1 second overlap.

In [ ]:
## CODE GOES HERE

**Exercise:** Create 4-second-long epochs with 25% overlap.

In [ ]:
## CODE GOES HERE

You may have noticed that when creating `Epochs` in this way, we do not specify the baseline settings.

Inspecting the `Epochs` shows that `baseline` is set to `"off"`.

In [ ]:
# Create continuous epochs
epochs = mne.make_fixed_length_epochs(raw=raw, duration=2)

# Show information about the epochs
epochs

This is no problem, as we can use the [`apply_baseline()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.apply_baseline) method of the `Epochs` object to do this.

In [ ]:
# Baseline-correct the epochs
epochs.apply_baseline(baseline=(None, None))

Above, we added a baseline based on the whole epoch duration.

But what would happen if we set the baseline to the [-1, 0] second window?

In [ ]:
try:
    epochs.apply_baseline(baseline=(-1, 0))
except ValueError as error:
    print(f"ValueError: {error}")

We get an error!

This is because `Epochs` created from `make_fixed_length_epochs()` always start at 0 seconds and end at the epoch's duration (in this case, 2 seconds).

**Exercises - Baseline-correcting `Epochs` object**

**Exercise:** Apply a baseline to the first second of the `Epochs` object.

There are two ways you can specify this, so try to include them both.

In [ ]:
## CODE GOES HERE

**Exercise:** Apply a baseline to the last second of the `Epochs` object.

Again, there are two ways you can specify this, so try to include them both.

In [ ]:
## CODE GOES HERE

As you can see, MNE has plenty of tools for creating epochs of data, either around event markers or as continuous segments of data.

### Part 3 - Creating `Epochs` from arrays

Just like for `Raw` objects, we can also create `Epochs` objects from data arrays. Specifically, we create [`mne.EpochsArray`](https://mne.tools/stable/generated/mne.EpochsArray.html) objects.

Again, this requires that we provide some metadata so that MNE can keep track of what the data represents. This is also done as an [`Info`](https://mne.tools/stable/generated/mne.Info.html) object.

Below, we randomly generate some data of 3 channels and 1,000 timepoints, then reshape this into 10 1-second-long epochs.

Remember that MNE expects epoched data to have shape `(epochs, channels, times)`.

In [ ]:
# Define parameters for generating data
n_channels = 3
n_epochs = 10
n_times = 1000  # samples
n_times_per_epoch = n_times // n_epochs  # samples
np.random.seed(44)  # set seed for consistency

# Generate the data
data = np.random.randn(n_channels, n_times)
data = np.reshape(data, (n_channels, n_epochs, n_times_per_epoch))
data = data.transpose((1, 0, 2))
print(f"Data has shape: {data.shape} (epochs, channels, times)")

**Exercises - Creating `Epochs` from arrays**

**Exercise:** Create an `Info` object for this data using the [`mne.create_info()`](https://mne.tools/stable/generated/mne.create_info.html) function.

Recall that we need to specify:
- the names of the channels - `ch_names` parameter
- the types of the channels - `ch_types` parameter
- the sampling frequency - `sfreq` parameter

Create the `Info` object for the 3 channels with: names of your choice; of type EEG; and a sampling frequency of 100 Hz.

In [ ]:
## CODE GOES HERE

We can then pass the data array and the `Info` object to the `EpochsArray` class.

In [ ]:
# Store the data and information in an EpochsArray object
epochs = mne.EpochsArray(data=data, info=info)

# Show what is stored in the EpochsArray object
epochs.plot(scalings="auto", n_epochs=3)
epochs

Just like an `Epochs` object, we can specify the baseline correction to apply (default is no baseline correction).

**Exercise:** Create an `EpochsArray` object from the data, with baseline correction for the whole epoch duration.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EpochsArray` object from the data, with baseline correction for the first half of each epoch.

In [ ]:
## CODE GOES HERE

Like when we created continuous epochs, the first sample of each epoch is considered to be 0 seconds.

We can change this when creating the `EpochsArray` object, similarly to how we specified `tmin` when creating `Epochs` objects from `Raw` objects.

**Exercise:** Create an `EpochsArray` object from the data with times in the window [-1, 0] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EpochsArray` object from the data with times in the window [-0.5, +0.5] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EpochsArray` object from the data with times in the window [-0.5, +0.5] seconds, and baseline correct it for the first half of the epochs.

In [ ]:
## CODE GOES HERE

## Conclusion

Alongside `Raw` objects, `Epochs` objects are some of the most heavily used parts of MNE, storing segments of data around experimentally-relevent events, or fixed-length chunks of continuous data.

They can be created from `Raw` objects (`Raw` -> `Epochs`), or from arrays (`array` -> `EpochsArray`).

In the upcoming notebooks, we will continue to build on this foundation for working with epochs as we explore different forms of data analysis.

## Additional resources

MNE tutorial on `Epochs` objects: https://mne.tools/stable/auto_tutorials/epochs/10_epochs_overview.html